In [1]:
from cyberbattle.agents.attacker.attacker_interface import Attacker, AttackerGoal

from cyberbattle.agents.battle_environment import CyberBattleEnv
from cyberbattle.env.samples.little_network import get_little_environment_network, Dev, DSI

num_client = 5
net = get_little_environment_network(num_client)
profiles = {
    DSI(num_client): 1,
    Dev(num_client): num_client - 1
}
env = CyberBattleEnv(net, profiles)

In [2]:
env.attacks.get_attacks_by_machines_string()

{'PC_1': {},
 'PC_2': {},
 'PC_3': {'System Owner/User Discovery': {'data sources triggered': 'Process',
   'phase name': 'discovery',
   'Type': 'Local'}},
 'PC_4': {},
 'PC_5': {'Password Cracking': {'data sources triggered': 'User Account',
   'phase name': 'credential-access',
   'Type': 'Remote'}},
 'Switch_1': {},
 'Switch_2': {},
 'Router': {},
 'DatabaseServer': {'Browser Session Hijacking': {'data sources triggered': 'Logon Session',
   'phase name': 'collection',
   'Type': 'Local'}},
 'MailServer': {'Network Service Scanning': {'data sources triggered': 'Cloud Service',
   'phase name': 'discovery',
   'Type': 'Local'}},
 'CommunicationServer': {},
 'GoogleDrive': {'Password Guessing': {'data sources triggered': 'User Account',
   'phase name': 'credential-access',
   'Type': 'Remote'}},
 'Firewall_1': {},
 'Firewall_2': {}}

In [3]:
attacker = Attacker(
    goals=AttackerGoal(
        reward=0,
        nb_flag=1
    ),
    attacks=env.id_to_attack,
    network=net,
    attacks_by_machine=env.attacks_by_machine,
    start_time=env.get_start_time()
)

print(attacker.get_discovered_machines())
attacker.attack_as_string()

['PC_3']


{0: 'System Owner/User Discovery',
 1: 'Password Cracking',
 2: 'Browser Session Hijacking',
 3: 'Network Service Scanning',
 4: 'Password Guessing'}

In [4]:
import numpy as np 

attack = {'local': np.array([0, 0])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

20 Is activity : True, source : PC_3, target : PC_3, port : sudo, data source triggered : Process, is error : 0


In [5]:
attacker.get_discovered_machines()

['PC_3', 'PC_1', 'PC_2', 'PC_4', 'PC_5']

In [6]:
attack = {'remote': np.array([0, 4, 1])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

25 Is activity : True, source : PC_3, target : PC_5, port : sudo, data source triggered : User Account, is error : 0


In [7]:
attacker.get_discovered_machines(), attacker.get_discovered_credentials()

(['PC_3', 'PC_1', 'PC_2', 'PC_4', 'PC_5', 'MailServer'],
 [('HTTPS', 'MailServer', 'DSI')])

In [8]:
attack = {'connect': np.array([0, 0])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

50 Is activity : True, source : PC_3, target : MailServer, port : HTTPS, data source triggered : User Account: User Account Authentification, is error : 0


In [9]:
attacker.get_infected_machines()

['PC_3', 'MailServer']

In [10]:
attack = {'local': np.array([5, 3])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

15 Is activity : True, source : MailServer, target : MailServer, port : HTTPS, data source triggered : Cloud Service, is error : 0


In [12]:
attacker.get_discovered_machines()

['PC_3',
 'PC_1',
 'PC_2',
 'PC_4',
 'PC_5',
 'MailServer',
 'DatabaseServer',
 'CommunicationServer',
 'GoogleDrive']

In [13]:
attack = {'remote': np.array([0, 8, 4])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

20 Is activity : True, source : PC_3, target : GoogleDrive, port : HTTPS, data source triggered : User Account, is error : 0


In [15]:
attacker.get_discovered_machines(), attacker.get_discovered_credentials()

(['PC_3',
  'PC_1',
  'PC_2',
  'PC_4',
  'PC_5',
  'MailServer',
  'DatabaseServer',
  'CommunicationServer',
  'GoogleDrive'],
 [('HTTPS', 'MailServer', 'DSI'), ('HTTPS', 'DatabaseServer', 'DSI')])

In [16]:
attack = {'connect': np.array([0, 1])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

50 Is activity : True, source : PC_3, target : DatabaseServer, port : HTTPS, data source triggered : User Account: User Account Authentification, is error : 0


In [23]:
attack = {'submarine'}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

0 Is activity : False, source : None, target : None, port : None, data source triggered : None, is error : None


In [18]:
attacker.get_infected_machines()

['PC_3', 'MailServer', 'DatabaseServer']

In [19]:
attack = {'local': np.array([6, 2])}
reward, activity = attacker.on_step(attack)

print(reward, activity.get_description())

1000 Is activity : True, source : DatabaseServer, target : DatabaseServer, port : HTTPS, data source triggered : Logon Session, is error : 0


In [20]:
attacker.get_captured_flag()

1

In [22]:
attacker.reached_goals()

True